In [1]:
import cv2
from imutils import paths
import numpy as np
import torch
import pyttsx3
from PIL import Image

# YOLOv5 모델 로드
model = torch.hub.load("ultralytics/yolov5", "yolov5s")

Using cache found in C:\Users\82102/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-9-10 Python-3.10.9 torch-2.0.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


# 단일 객체-웹캠

In [3]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s') 
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FPS, 30)

know_distance = 300
know_width = 180

GREEN = (0,255,0)
RED = (0,0,255)
WHITE =(255,255,255)
fonts = cv2.FONT_HERSHEY_COMPLEX

def FocalLength(measured_distance, real_width, width_in_rf_image):
    focal_length = (width_in_rf_image * measured_distance)/ real_width
    return focal_length

def Distance_finder (Focal_length, real_face_width, face_width_in_frame):
    distance = (real_face_width * Focal_length)/face_width_in_frame
    return distance

def face_data(image):
    face_width = 0
    
    results = model(image)
    predictions = results.pred[0]
    
    for detection in results.xyxy[0]:
        x_min, y_min, x_max, y_max = map(int, detection[:4])
        face_width = x_max-x_min
    return face_width

ref_image = cv2.imread('my_image/car1.jpg')
ref_image_face_width = face_data(ref_image)  #frame 너비
Focal_length_found = FocalLength(know_distance,know_width,ref_image_face_width)
#print(Focal_length_found)
#cv2.imshow("Me.jpg",ref_image)
 
while True:
    _, frame = cap.read()
    
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = model(frame_rgb)
          
    for detection in results.xyxy[0]:
        face_width_in_frame = 0
        x_min, y_min, x_max, y_max = map(int, detection[:4])
        face_width_in_frame = x_max-x_min
        real_width = (face_width_in_frame/ref_image_face_width)*know_width 

        if face_width_in_frame != 0:
            Distance = Distance_finder(Focal_length_found,know_width,face_width_in_frame)

        #cv2.putText(frame,f"Distance = {Distance}",(detection[2]+10,detection[3]),fonts,0.6,(WHITE),2)

        center_x = (x_min + x_max) // 2
        center_y = (y_min + y_max) // 2

        cv2.putText(frame,f"Distance = {Distance:.2f}" , (center_x, center_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

    #frame = results.render()[0]
    cv2.imshow("frame",frame)

    if cv2.waitKey(1) == ord("q"):
        break 

cap.release()
cv2.destroyAllWindows()

Using cache found in C:\Users\82102/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-9-10 Python-3.10.9 torch-2.0.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


# 단일 객체 - 이미지

In [11]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s') 
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FPS, 30)

know_distance = 300
know_width = 180

GREEN = (0,255,0)
RED = (0,0,255)
WHITE =(255,255,255)
fonts = cv2.FONT_HERSHEY_COMPLEX

def FocalLength(measured_distance, real_width, width_in_rf_image):
    focal_length = (width_in_rf_image * measured_distance)/ real_width
    return focal_length

def Distance_finder (Focal_length, real_face_width, face_width_in_frame):
    distance = (real_face_width * Focal_length)/face_width_in_frame
    return distance

def face_data(image):
    face_width = 0
    
    results = model(image)
    predictions = results.pred[0]
    
    for detection in results.xyxy[0]:
        x_min, y_min, x_max, y_max = map(int, detection[:4])
        face_width = x_max-x_min
    return face_width

def resize_image(image, target_width, target_height):
    return cv2.resize(image, (target_width, target_height))


ref_image = cv2.imread('my_image/car1.jpg')
my_image = cv2.imread('my_image/car4.jpg')

if ref_image.shape[0] < my_image.shape[0] or ref_image.shape[1] < my_image.shape[1]:
    my_image = resize_image(my_image, ref_image.shape[1], ref_image.shape[0])
else:
    ref_image = resize_image(ref_image, my_image.shape[1], my_image.shape[0])

ref_image_face_width = face_data(ref_image)


ref_image_face_width = face_data(ref_image)  #frame 너비
Focal_length_found = FocalLength(know_distance,know_width,ref_image_face_width)

#cv2.imshow("jun2.png",ref_image)


results = model(my_image)

          
for detection in results.xyxy[0]:
    face_width_in_frame = 0
    x_min, y_min, x_max, y_max = map(int, detection[:4])
    face_width_in_frame = x_max-x_min
    real_width = (face_width_in_frame/ref_image_face_width)*know_width 

    if face_width_in_frame != 0:
        Distance = Distance_finder(Focal_length_found,know_width,face_width_in_frame)

    print(Distance)
    center_x = (x_min + x_max) // 2
    center_y = (y_min + y_max) // 2

    cv2.putText(my_image,f"Distance = {Distance:.2f}" , (center_x, center_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

    #frame = results.render()[0]
    cv2.imshow("frame",my_image)




# face_width_in_frame = face_data(my_image)

# if face_width_in_frame != 0:
#     Distance = Distance_finder(Focal_length_found,know_width,face_width_in_frame)

#     cv2.putText(my_image,f"Distance = {Distance}",(50,50),fonts,0.6,(GREEN),2)

# cv2.imshow("car2.jpg",my_image)

cv2.waitKey(0)
cv2.destroyAllWindows()



Using cache found in C:\Users\82102/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-9-10 Python-3.10.9 torch-2.0.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


412.5
113.51351351351352
133.52601156069363


# 다중 객체 - 웹캠

In [4]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s') 
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FPS, 30)

know_distance_1 = 30
know_width_1 = 15

know_distance_2 = 40
know_width_2 = 18

GREEN = (0,255,0)
RED = (0,0,255)
WHITE =(255,255,255)
fonts = cv2.FONT_HERSHEY_COMPLEX


def FocalLength(measured_distance, real_width, width_in_rf_image):
    focal_length = (width_in_rf_image * measured_distance)/ real_width
    return focal_length

def Distance_finder (Focal_length, real_face_width, face_width_in_frame):
    distance = (real_face_width * Focal_length)/face_width_in_frame
    return distance

def face_data(image):
    face_width = 0
    
    results = model(image)
    predictions = results.pred[0]
    
    for detection in results.xyxy[0]:
        x_min, y_min, x_max, y_max = map(int, detection[:4])
        face_width = x_max-x_min
    return face_width

ref_image_1 = cv2.imread('my_image/phone.jpg')
ref_image_face_width_1 = face_data(ref_image_1)  #frame 너비
Focal_length_found_1 = FocalLength(know_distance_1,know_width_1,ref_image_face_width_1)

ref_image_2 = cv2.imread('my_image/Me.jpg')
ref_image_face_width_2 = face_data(ref_image_2)  #frame 너비
Focal_length_found_2 = FocalLength(know_distance_2,know_width_2,ref_image_face_width_2)
#print(Focal_length_found)
#cv2.imshow("Me.jpg",ref_image)
 
while True:
    _, frame = cap.read()
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = model(frame_rgb)
    class_labels = results.names

    for detection in results.xyxy[0]:
        face_width_in_frame = 0
        x_min, y_min, x_max, y_max = map(int, detection[:4])
        face_width_in_frame = x_max-x_min
        class_index = int(detection[5])
        class_name = class_labels[class_index]
        
        if (class_name == "person"):
            ref_image_face_width = ref_image_face_width_2
            know_width = know_width_2
            Focal_length_found = Focal_length_found_2
        else :
            ref_image_face_width = ref_image_face_width_1
            know_width = know_width_1
            Focal_length_found = Focal_length_found_1

        real_width = (face_width_in_frame/ref_image_face_width)*know_width 

        if face_width_in_frame != 0:
            Distance = Distance_finder(Focal_length_found,know_width,face_width_in_frame)

        #cv2.putText(frame,f"Distance = {Distance}",(detection[2]+10,detection[3]),fonts,0.6,(WHITE),2)

        center_x = (x_min + x_max) // 2
        center_y = (y_min + y_max) // 2

        cv2.putText(frame,f"Distance = {Distance:.2f}" , (center_x, center_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

    #frame = results.render()[0]
    cv2.imshow("frame",frame)

    if cv2.waitKey(1) == ord("q"):
        break 

cap.release()
cv2.destroyAllWindows()

Using cache found in C:\Users\82102/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-9-10 Python-3.10.9 torch-2.0.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


# gpt

# 17 

In [1]:
import random

class VacuumCleaner:
    def __init__(self, rooms):
        self.rooms = rooms  # 방의 청결 상태 (True: 더러움, False: 깨끗)
        self.position = random.choice([0, 1])  # 에이전트 초기 위치 (0 또는 1)
        self.score = 0  # 성능 점수

    def clean(self):
        self.rooms[self.position] = False  # 현재 위치의 방을 청소
        self.score += 1  # 점수 증가

    def move(self):
        self.position = 1 - self.position  # 다른 방으로 이동

    def act(self):
        if self.rooms[self.position]:  # 현재 방이 더러우면
            self.clean()
        else:
            self.move()

def run_simulation():
    num_episodes = 1000  # 실행할 에피소드 수
    total_score = 0

    for _ in range(num_episodes):
        rooms = [random.choice([True, False]) for _ in range(2)]  # 초기 방 상태 설정
        vacuum = VacuumCleaner(rooms)
        while any(rooms):  # 모든 방이 깨끗해질 때까지 반복
            vacuum.act()

        total_score += vacuum.score

    average_score = total_score / num_episodes
    return average_score

if __name__ == "__main__":
    avg_score = run_simulation()
    print(f"평균 성능 점수: {avg_score}")


평균 성능 점수: 0.994


# 16

In [2]:
import random

class VacuumCleanerSensor:
    def __init__(self, room):
        self.room = room

    def check_dirt(self):
        return random.choice([True, False])  # 방의 먼지 상태를 무작위로 시뮬레이션

class VacuumCleanerActuator:
    def move(self, vacuum, to_room):
        print(f"진공 청소기가 방 {to_room}로 이동합니다.")
        vacuum.position = to_room

    def clean(self, vacuum):
        print(f"진공 청소기가 방 {vacuum.position}을 청소합니다.")
        vacuum.clean()

class VacuumCleanerEnvironment:
    def __init__(self, num_rooms=2):
        self.num_rooms = num_rooms
        self.rooms = [random.choice([True, False]) for _ in range(num_rooms)]  # 초기 방 상태 설정

class VacuumCleaner:
    def __init__(self, sensor, actuator, environment):
        self.position = 0  # 초기 위치
        self.score = 0  # 성능 점수
        self.sensor = sensor
        self.actuator = actuator
        self.environment = environment

    def clean(self):
        self.score += 1

    def run(self):
        for i in range(self.environment.num_rooms):
            print(f"\n현재 위치: 방 {self.position}")
            dirty = self.sensor.check_dirt()
            if dirty:
                self.actuator.clean(self)
                print(f"현재 성능 점수: {self.score}")
            else:
                next_room = (self.position + 1) % self.environment.num_rooms
                self.actuator.move(self, next_room)

if __name__ == "__main__":
    num_rooms = 2
    sensor = VacuumCleanerSensor(num_rooms)
    actuator = VacuumCleanerActuator()
    environment = VacuumCleanerEnvironment(num_rooms)
    vacuum = VacuumCleaner(sensor, actuator, environment)

    print("진공 청소기 시뮬레이션 시작...\n")
    vacuum.run()


진공 청소기 시뮬레이션 시작...


현재 위치: 방 0
진공 청소기가 방 1로 이동합니다.

현재 위치: 방 1
진공 청소기가 방 1을 청소합니다.
현재 성능 점수: 1


In [5]:
import random

class Agent:
    def __init__(self):
        self.state = None  # 에이전트의 상태
        self.actions = ["action1", "action2", "action3"]  # 가능한 행동 목록

    def set_state(self, state):
        self.state = state

    def choose_action(self):
        # 여기에 행동 선택 로직을 구현
        # 현재 상태(state)를 기반으로 행동 선택
        return random.choice(self.actions)  # 무작위 행동 선택

# 에이전트 인스턴스 생성
agent = Agent()

# 에이전트 성능 측정을 위한 실험
num_episodes = 100  # 에피소드 수
total_rewards = 0

for episode in range(num_episodes):
    # 에피소드 시작 시 초기 상태 설정
    initial_state = random.randint(1, 10)  # 간단한 예제를 위해 1에서 10까지의 상태로 가정
    agent.set_state(initial_state)

    episode_reward = 0  # 현재 에피소드의 보상 초기화

    # 에피소드 진행
    for step in range(10):  # 각 에피소드에서 10단계만 진행
        action = agent.choose_action()  # 에이전트가 행동 선택
        # 여기에서 환경과의 상호작용 또는 보상 계산 로직을 추가할 수 있습니다.

        # 예제에서는 간단하게 무작위로 보상을 설정
        reward = random.randint(0, 10)
        episode_reward += reward

    total_rewards += episode_reward

# 평균 보상 계산
average_reward = total_rewards / num_episodes
print(f"평균 보상: {average_reward}")


평균 보상: 50.32
